In [1]:
import re
import datetime
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
##local_path = "/your/path/here"
local_path = "/users/thudson/Documents/Dropbox/chicago-police-data/02-column_std"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/output')

['incomplete',
 'TRRs',
 'complaints-cpd-2016-dec',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-nov',
 'complaints-cpd-2016-oct',
 'complaints-ipra-2016-apr',
 'complaints-merged-2015',
 'shootings-cpd-feb2016',
 'shootings-ipra-may2016']

In [5]:
in_path = path + '/output/complaints-cpd-2016-oct/'
out_path = local_path + '/output/complaints-cpd-2016-oct/'

In [ ]:
os.mkdir(out_path)

In [16]:
##doc_path = '/Data/chicago-police-data/02-column_std'
##column_dict = db.download_file(doc_path+'/doc/','Column_Dictionary.csv')

doc_path = local_path
column_dict = pd.read_csv(doc_path+'/doc/'+'Column_Dictionary.csv').iloc[:,0:2]
column_dict.head()

,Orig Column Name,New Name
0,Victim_Age,Civilian.Victim.Age
1,Victim_Gender,Civilian.Victim.Gender
2,Victim_Race,Civilian.Victim.Race_Code
3,Witness_Age,Civilian.Witness.Age
4,Witness_Gender,Civilian.Witness.Gender


In [17]:
## SOMETHING TO PUT IN UTILS? HOW WOULD THE SPECIFIED COLUMN_DICT WORK?

def column_mapper(df,
                  column_dict = column_dict):
    current_columns = pd.DataFrame(df.columns.tolist(),columns=['Orig Column Name'])
    new_columns = current_columns.merge(column_dict,how='left',on='Orig Column Name')['New Name'].tolist() 
    if current_columns.shape[0] == len(new_columns):
        return new_columns
    else:
        print("Join failed, more columns returned than expected")

## October 2016 Data

### Report 1

In [18]:
in_path_oct_2016_report = in_path
out_path_oct_2016_report = out_path


files = db.list_files(in_path_oct_2016_report)
files

['CR_AllRecords.csv', 'CR_AllRecords.xlsx', 'CR_AllRecords_metadata.csv']

In [19]:
file = files[0]
df = db.download_file(in_path_oct_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-cpd-2016-oct/cr_allrecords.csv


/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['Officer_Name',
 'Officer_Unit',
 'Date_of_Appointment',
 'CRID',
 'Initial_Complaint_Category',
 'Final_Complaint_Category',
 'Incident_Date',
 'Complaint_Date',
 'Closed_Date',
 'Final_Finding',
 'Final_Action_Taken',
 'Final_Action_Description']

In [20]:
def Complaint_Category_Splitter(string):
    new_string = str(string).split('   ',1)
    if len(new_string)==1:
        new_string= ['nan','nan']
    return new_string

split_df_initial = pd.DataFrame(df.Initial_Complaint_Category.apply(Complaint_Category_Splitter).tolist(),
                                columns = ['Initial_Complaint_Category','Initial_Complaint_Description'])

split_df_final = pd.DataFrame(df.Final_Complaint_Category.apply(Complaint_Category_Splitter).tolist(),
                                columns = ['Final_Complaint_Category','Final_Complaint_Description'])


df = df.drop(["Initial_Complaint_Category","Final_Complaint_Category"],axis=1)
df1 = df.merge(split_df_initial,how='left',left_index=True,right_index=True)
df2 = df1.merge(split_df_final,how='left',left_index=True,right_index=True)
print(df.shape)
print(df2.shape)
df2.head()

(134683, 10)
(134683, 14)


,Officer_Name,Officer_Unit,Date_of_Appointment,CRID,Incident_Date,Complaint_Date,Closed_Date,Final_Finding,Final_Action_Taken,Final_Action_Description,Initial_Complaint_Category,Initial_Complaint_Description,Final_Complaint_Category,Final_Complaint_Description
0,"ABBATE, ANTHONY G",011,05-DEC-1994,237004,21-MAY-1997,21-MAY-1997,25-MAY-1999,UN,600,NO ACTION TAKEN,05A,ARRESTEE - DURING ARREST,05B,ARRESTEE - DURING ARREST
1,"ABBATE, ANTHONY G",011,05-DEC-1994,258124,19-NOV-1999,20-NOV-1999,07-DEC-2000,NS,600,NO ACTION TAKEN,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT
2,"ABBATE, CARMEL G",641,06-JAN-1969,190369,11-FEB-1992,11-FEB-1992,09-APR-1992,UN,600,NO ACTION TAKEN,03G,MISCELLANEOUS,03G,MISCELLANEOUS
3,"ABBATE, CARMEL G",640,06-JAN-1969,210426,30-JUL-1994,14-JUL-1994,01-DEC-1994,NS,600,NO ACTION TAKEN,10V,INVENTORY PROCEDURES,04E,INVENTORY PROCEDURES
4,"ABBATE, CARMEL G",640,06-JAN-1969,219424,12-JUL-1995,19-JUL-1995,23-JAN-1996,NS,600,NO ACTION TAKEN,09J,MISCELLANEOUS,05A,MISCELLANEOUS


In [21]:
df = df2

In [22]:
df.columns = column_mapper(df)
df.columns.tolist()

['Officer.Accused.Full_Name',
 'Officer.Accused.Unit',
 'Officer.Accused.Appointed_Date',
 'CRID',
 'Incident.Time.Date',
 'Complaint.Date',
 'Complaint.Current.Closed_date',
 'Complaint.Final.Finding',
 'Complaint.Final.Action_Taken',
 'Complaint.Final.Action_Description',
 'Complaint.Initial.Category',
 'Complaint.Initial.Description',
 'Complaint.Final.Category',
 'Complaint.Final.Description']

In [25]:
df['Complaint.Final.Action_Description']

0              NO ACTION TAKEN
1              NO ACTION TAKEN
2              NO ACTION TAKEN
3              NO ACTION TAKEN
4              NO ACTION TAKEN
5              NO ACTION TAKEN
6              NO ACTION TAKEN
7              NO ACTION TAKEN
8                    REPRIMAND
9              NO ACTION TAKEN
10             NO ACTION TAKEN
11             NO ACTION TAKEN
12             NO ACTION TAKEN
13             NO ACTION TAKEN
14             NO ACTION TAKEN
15             NO ACTION TAKEN
16             NO ACTION TAKEN
17                  SEPARATION
18             NO ACTION TAKEN
19             NO ACTION TAKEN
20             NO ACTION TAKEN
21             NO ACTION TAKEN
22             NO ACTION TAKEN
23                   REPRIMAND
24             NO ACTION TAKEN
25             NO ACTION TAKEN
26        SUSTAINED-NO PENALTY
27             NO ACTION TAKEN
28                   REPRIMAND
29             NO ACTION TAKEN
                  ...         
134653         NO ACTION TAKEN
134654  

In [23]:
df.to_csv(out_path_oct_2016_report+file,index=False)